In [1]:
# Load the SQL magic extension
# https://github.com/ploomber/jupysql
# this extension allows us to connect to DBs and issue SQL commands
%load_ext sql

In [36]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text, inspect

In [37]:
engine = create_engine('sqlite:///kbo_database.db')
print(type(engine))

<class 'sqlalchemy.engine.base.Engine'>


In [40]:
Session = sessionmaker(bind=engine)
session = Session()

In [41]:
# Code to drop belgium_regions table if it exists

# with engine.connect() as conn:
#     conn.execute(text("DROP TABLE IF EXISTS belgium_regions"))
#     conn.commit()

# inspector = inspect(engine)
# print(inspector.get_table_names())

In [42]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class BelgiumRegion(Base):
    __tablename__ = "belgium_regions"

    id = Column(Integer, primary_key=True)
    region_code = Column(String, nullable=False)
    post_code = Column(String, nullable=False)
    region_name_fr = Column(String, nullable=False)


Base.metadata.create_all(engine)
inspector = inspect(engine)
print(inspector.get_table_names())


['activity', 'address', 'belgium_regions', 'branch', 'code', 'contact', 'denomination', 'enterprise', 'establishment', 'meta']


/var/folders/n4/xhm5c1x102q3rdg91bgwysbw0000gn/T/ipykernel_90583/40714184.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [43]:
# Create new table for Belgium regions. Only run once
import csv

with open("georef-belgium-postal-codes.csv", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile, delimiter=";")
    for row in reader:
        entry = BelgiumRegion(
            region_code=row["Région code"],
            post_code=row["Post code"],
            region_name_fr=row["Région name (French)"]
        )
        session.add(entry)

session.commit()
session.close()

In [44]:
inspector = inspect(engine)
print(inspector.get_table_names())

['activity', 'address', 'belgium_regions', 'branch', 'code', 'contact', 'denomination', 'enterprise', 'establishment', 'meta']


In [45]:
def execute_query(query):
    with Session() as session:
        result = session.execute(text(query))
        for row in result:
            print(row)

In [47]:

inspector = inspect(engine)
print(inspector.get_columns("enterprise"))
# Add a new column to the enterprise table (RUN ONCE)
with Session() as session:
    session.execute(text("""
    ALTER TABLE enterprise
    ADD COLUMN StartDate_ISO TEXT;
    """))
    session.commit()

[{'name': 'EnterpriseNumber', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'Status', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'JuridicalSituation', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'TypeOfEnterprise', 'type': INTEGER(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'JuridicalForm', 'type': REAL(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'JuridicalFormCAC', 'type': REAL(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'StartDate', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'StartDate_ISO', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}]


OperationalError: (sqlite3.OperationalError) duplicate column name: StartDate_ISO
[SQL: 
    ALTER TABLE enterprise
    ADD COLUMN StartDate_ISO TEXT;
    ]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [48]:
with Session() as session:
    # Update the StartDate_ISO column with the formatted date
    session.execute(text("""
    UPDATE enterprise
    SET StartDate_ISO = substr(StartDate, 7, 4) || '-' || substr(StartDate, 4, 2) || '-' || substr(StartDate, 1, 2);
    """))
    session.commit() 



In [49]:
# Regional distribution of BELGIAN enterprises
query1= """
SELECT
	COUNT(DISTINCT EnterpriseNumber) AS enterprise_count,
	region_code,
	region_name_fr
FROM
	enterprise
	INNER JOIN address ON address.EntityNumber = enterprise.EnterpriseNumber
	INNER JOIN belgium_regions ON address.Zipcode = belgium_regions.post_code
WHERE region_code IS NOT NULL
    AND region_code != ''
GROUP BY
	region_code
ORDER BY enterprise_count DESC
"""
execute_query(query1)

(647538, '02000', 'Région flamande')
(247945, '03000', 'Région wallonne')
(175914, '01000', 'Région de Bruxelles-Capitale')


In [55]:
# distribution of activity groups
query2 = """
SELECT
    COUNT(activity.EntityNumber) AS count,
    activity.ActivityGroup as "Activity Group",
    code.Description as "Description"
FROM
    activity
    LEFT JOIN code ON code.Code = activity.ActivityGroup 
        AND code.Category = 'ActivityGroup'
        AND code.Language = 'NL'
GROUP BY
    activity.ActivityGroup, code.Description
ORDER BY count DESC
"""

execute_query(query2)

(28855664, 3, 'Activiteiten')
(6487053, 1, 'BTW-activiteiten')
(1219587, 6, 'RSZ-activiteiten')
(25800, 7, 'Activiteiten gesubsideerd onderwijs')
(6964, 5, 'RSZPPO-activiteiten')
(2423, 4, 'Activiteiten federaal openbaar ambt')


In [56]:
# distribution of juridical form
query3 = """
SELECT
    COUNT(enterprise.JuridicalForm) AS entity_count,
    Description,
    enterprise.JuridicalForm
FROM
    enterprise
    LEFT JOIN code ON code.Code = enterprise.JuridicalForm 
        AND code.Category = 'JuridicalForm'
GROUP BY
    enterprise.JuridicalForm
HAVING entity_count > 0
ORDER BY entity_count DESC
"""

execute_query(query3)

(1480152, 'Besloten Vennootschap', 610.0)
(417639, 'Association sans but lucratif', 17.0)
(325335, 'Association des copropriétaires', 70.0)
(259623, 'Besloten vennootschap met beperkte aansprakelijkheid', 15.0)
(242025, 'Aktiengesellschaft', 14.0)
(202728, 'Ausländische Einheit', 30.0)
(142056, 'Commanditaire vennootschap', 612.0)
(83211, 'Gesellschaften oder Vereinigungen ohne Rechtspersönlichkeit', 721.0)
(74664, 'Offene Handelsgesellschaft', 11.0)
(48546, 'Einfache Kommanditgesellschaft', 12.0)
(36447, 'Ausländische Einheit ohne Niederlassung in Belgien mit einem Fiskalvertreter für die MwSt.', 235.0)
(24960, 'BTW-eenheid', 3.0)
(21567, 'Coöperatieve vennootschap (oud statuut)', 16.0)
(14610, 'Etablissement public', 124.0)
(12333, 'Coöperatieve vennootschap met beperkte aansprakelijkheid', 8.0)
(11694, 'Ausländische Einheit mit einer Immobilie in Belgien (mit Rechtspersönlichkeit)', 230.0)
(8586, 'Coöperatieve vennootschap met onbeperkte aansprakelijkheid', 6.0)
(7506, 'Association 

In [100]:
# Average age of enterprises by Nace code
query4 = """
SELECT
    NaceCode,
    AVG((julianday('now') - julianday(StartDate_ISO)) / 365.25) AS average_age_years
FROM
    enterprise
    INNER JOIN activity ON activity.EntityNumber = enterprise.EnterpriseNumber
GROUP BY
    NaceCode
ORDER BY average_age_years DESC
LIMIT 100
"""
execute_query(query4)

(80226, 96.48851370646106)
(75302, 81.02048641091837)
(10100, 68.73875326840493)
(70202, 66.53313188974334)
(75114, 62.9098689426007)
(66033, 60.477692677159546)
(26240, 60.13669988031459)
(84114, 59.91288191270181)
(11100, 59.82937612945901)
(75220, 57.56448906580398)
(91310, 56.75960119689872)
(84232, 56.10247674547544)
(75232, 56.10247674547544)
(80224, 55.94799014860153)
(65222, 55.57684319497308)
(75115, 55.23571881711587)
(26402, 54.489882631854634)
(75303, 54.36300946133604)
(80103, 54.31739803226531)
(84115, 53.87206047720787)
(75111, 53.50537637835453)
(66031, 52.35598461679645)
(80105, 52.19100058759272)
(66020, 51.79740194348072)
(41000, 51.61694591438872)
(1412, 51.267432255400145)
(80214, 50.79797903947997)
(27441, 50.72259994876086)
(24110, 50.69279470951003)
(27451, 50.66620022254594)
(85324, 50.21414644108622)
(94910, 50.13888615860418)
(91200, 50.12719488890266)
(55210, 49.63786346689912)
(26810, 49.30389544997423)
(85313, 49.246150327833504)
(29130, 49.241618233692904

In [58]:
# Percentage of enterprises by juridical form
query5 = """
SELECT
    COUNT(*) AS entity_count,
    Description,
    enterprise.JuridicalForm,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM enterprise WHERE JuridicalForm IS NOT NULL), 2) AS percentage
FROM
    enterprise
    LEFT JOIN code ON code.Code = enterprise.JuridicalForm 
        AND code.Category = 'JuridicalForm'
        AND code.Language = 'NL'  
WHERE enterprise.JuridicalForm IS NOT NULL
GROUP BY
    enterprise.JuridicalForm, Description
HAVING entity_count > 0
ORDER BY entity_count DESC
"""

execute_query(query5)

(493384, 'Besloten Vennootschap', 610.0, 42.78)
(139213, 'Vereniging zonder winstoogmerk', 17.0, 12.07)
(108445, 'Vereniging van mede-eigenaars', 70.0, 9.4)
(86541, 'Besloten vennootschap met beperkte aansprakelijkheid', 15.0, 7.5)
(80675, 'Naamloze vennootschap', 14.0, 7.0)
(67576, 'Buitenlandse entiteit', 30.0, 5.86)
(47352, 'Commanditaire vennootschap', 612.0, 4.11)
(27737, 'Vennootschap of vereniging zonder rechtspersoonlijkheid', 721.0, 2.41)
(24888, 'Vennootschap onder firma', 11.0, 2.16)
(16182, 'Gewone commanditaire vennootschap', 12.0, 1.4)
(12149, 'Buitenlandse entiteit zonder Belgische vestiging met vertegenwoordiger voor BTW', 235.0, 1.05)
(8320, 'BTW-eenheid', 3.0, 0.72)
(7189, 'Coöperatieve vennootschap (oud statuut)', 16.0, 0.62)
(4870, 'Openbare instelling', 124.0, 0.42)
(4111, 'Coöperatieve vennootschap met beperkte aansprakelijkheid', 8.0, 0.36)
(3898, 'Buitenlandse entiteit met onroerend goed in België (met rechtspersoonlijkheid)', 230.0, 0.34)
(2862, 'Coöperatieve v

In [59]:
# Distribution of companies outside Belgium
query6 = """
SELECT COUNT(*) as count, CountryNL
FROM address
where Zipcode IS NULL
GROUP BY CountryNL
ORDER BY count DESC
"""
execute_query(query6)

(2209, 'Duitsland (Bondsrep.)')
(1988, 'Groot-Brittannië')
(1291, 'Luxemburg (Groot-Hertogdom)')
(1156, 'Verenigde Staten van Amerika')
(720, 'Zwitserland')
(654, 'Frankrijk')
(336, 'Italië')
(284, 'Nederland')
(179, 'Liechtenstein')
(178, 'Ierland /Eire/')
(166, 'Spanje')
(162, 'Zweden')
(110, 'Panama')
(95, 'Maagdeneilanden')
(94, 'Denemarken')
(88, 'Japan')
(83, 'Oostenrijk')
(79, 'Canada')
(77, 'Portugal')
(74, 'Finland')
(72, 'Gibraltar')
(67, 'Nederlandse Antillen')
(67, 'Congo (Dem. Rep.)')
(54, 'Noorwegen')
(32, 'Liberia')
(31, 'Zuid-Afrika /Rep./')
(31, 'Israël')
(27, 'Griekenland')
(26, 'Onbepaald')
(22, 'Cyprus')
(20, 'Duitsland (Dem. rep.)')
(20, 'Australië')
(19, 'Monaco /Vorstendom/')
(18, 'Turkije')
(17, 'India')
(17, 'Bahamas')
(16, 'Marokko')
(15, 'Roemenië')
(14, 'Joegoslavië (Servië-Montenegro)')
(13, 'Libanon')
(13, 'Hong Kong(V.K.)')
(12, 'Polen ( Rep. )')
(9, 'Tunesië')
(9, 'China /Volksrep./')
(9, 'Andorra')
(8, 'Singapore')
(8, 'Nieuw-Zeeland')
(8, 'Malta')
(7, 

In [60]:
# Count of enterprises by starting month
query7 = """
    SELECT 
    	COUNT(*) AS count,  
    	strftime('%m', StartDate_ISO) AS month
    FROM enterprise
    GROUP BY month
    ORDER BY count DESC
"""
execute_query(query7)

(262813, '01')
(220905, '10')
(190441, '04')
(172372, '07')
(152819, '12')
(144267, '02')
(140084, '03')
(130018, '09')
(129974, '05')
(129013, '06')
(128929, '11')
(124611, '08')


In [61]:
# nr of enterprises created by year
query8 = """
    SELECT 
    	COUNT(*) AS count,  
    	strftime('%Y', StartDate_ISO) AS year
    FROM enterprise
    GROUP BY year
    ORDER BY count DESC
"""
execute_query(query8)

(130923, '2024')
(115432, '2023')
(105626, '2022')
(100446, '2021')
(84786, '2020')
(81171, '2019')
(74092, '2025')
(73309, '2018')
(66413, '2017')
(61028, '2016')
(52542, '2015')
(49466, '2014')
(44352, '2011')
(43779, '2013')
(42024, '2012')
(41178, '2010')
(40525, '1995')
(37958, '2007')
(37820, '2008')
(36784, '2009')
(34056, '2006')
(30255, '2005')
(27495, '2004')
(24043, '2002')
(23838, '2003')
(23660, '2001')
(23337, '2000')
(23217, '1998')
(22421, '1999')
(21951, '1996')
(21754, '1997')
(20339, '1992')
(19847, '1989')
(19358, '1993')
(19129, '1994')
(18941, '1991')
(18477, '1990')
(17549, '1988')
(15386, '1987')
(13429, '1986')
(12663, '1968')
(11487, '1985')
(10515, '1983')
(10428, '1984')
(9013, '1971')
(8816, '1982')
(7598, '1981')
(7350, '1980')
(6898, '1978')
(6810, '1979')
(6752, '1977')
(6267, '1976')
(5776, '1800')
(5531, '1975')
(4884, '1974')
(4643, '1973')
(4053, '1972')
(3624, '1969')
(3503, '1970')
(2422, '1967')
(2019, '1966')
(1974, '1965')
(1629, '1964')
(1436, 

In [68]:
# Company creation trends - year over year growth
query9 = """
SELECT 
    year,
    count AS companies_created,
    LAG(count) OVER (ORDER BY year) AS previous_year,
    ROUND(((count - LAG(count) OVER (ORDER BY year)) * 100.0 / 
           LAG(count) OVER (ORDER BY year)), 2) AS growth_percentage
FROM (
    SELECT 
        strftime('%Y', StartDate_ISO) AS year,
        COUNT(*) AS count
    FROM enterprise
    WHERE StartDate_ISO IS NOT NULL
    GROUP BY year
    ORDER BY year
) 
WHERE year >= '2005' and year < '2025'
ORDER BY growth_percentage DESC
"""

execute_query(query9)

('2021', 100446, 84786, 18.47)
('2016', 61028, 52542, 16.15)
('2024', 130923, 115432, 13.42)
('2014', 49466, 43779, 12.99)
('2006', 34056, 30255, 12.56)
('2010', 41178, 36784, 11.95)
('2007', 37958, 34056, 11.46)
('2019', 81171, 73309, 10.72)
('2018', 73309, 66413, 10.38)
('2023', 115432, 105626, 9.28)
('2017', 66413, 61028, 8.82)
('2011', 44352, 41178, 7.71)
('2015', 52542, 49466, 6.22)
('2022', 105626, 100446, 5.16)
('2020', 84786, 81171, 4.45)
('2013', 43779, 42024, 4.18)
('2008', 37820, 37958, -0.36)
('2009', 36784, 37820, -2.74)
('2012', 42024, 44352, -5.25)
('2005', 30255, None, None)


In [69]:
# Debug: Check what types of address situations exist
debug_query = """
SELECT 
    'Enterprises with Belgian addresses' as category,
    COUNT(DISTINCT enterprise.EnterpriseNumber) as count
FROM enterprise
INNER JOIN address ON enterprise.EnterpriseNumber = address.EntityNumber
WHERE address.Zipcode IS NOT NULL

UNION ALL

SELECT 
    'Enterprises with foreign addresses' as category,
    COUNT(DISTINCT enterprise.EnterpriseNumber) as count
FROM enterprise
INNER JOIN address ON enterprise.EnterpriseNumber = address.EntityNumber
WHERE address.Zipcode IS NULL

UNION ALL

SELECT 
    'Enterprises with NO address records' as category,
    COUNT(DISTINCT enterprise.EnterpriseNumber) as count
FROM enterprise
LEFT JOIN address ON enterprise.EnterpriseNumber = address.EntityNumber
WHERE address.EntityNumber IS NULL

UNION ALL

SELECT 
    'Total enterprises' as category,
    COUNT(DISTINCT EnterpriseNumber) as count
FROM enterprise
"""

execute_query(debug_query)

('Enterprises with Belgian addresses', 1142297)
('Enterprises with foreign addresses', 10904)
('Enterprises with NO address records', 773045)
('Total enterprises', 1926246)


In [94]:
# Nace code distribution for enterprises abroad
query11 = """
SELECT 
    NaceCode, 
    COUNT(DISTINCT activity.EntityNumber) as enterprise_count,
    ROUND(COUNT(DISTINCT activity.EntityNumber) * 100.0 / (
        SELECT COUNT(DISTINCT activity.EntityNumber) 
        FROM address 
        INNER JOIN activity ON activity.EntityNumber = address.EntityNumber 
        WHERE Zipcode IS NULL
    ), 2) AS percentage
FROM address
INNER JOIN activity ON activity.EntityNumber = address.EntityNumber
WHERE Zipcode IS NULL
GROUP BY NaceCode
ORDER BY enterprise_count DESC
"""
execute_query(query11)

(51550, 16, 7.44)
(46851, 16, 7.44)
(46751, 16, 7.44)
(94110, 13, 6.05)
(91110, 12, 5.58)
(99000, 9, 4.19)
(46720, 9, 4.19)
(51872, 8, 3.72)
(51460, 8, 3.72)
(46460, 8, 3.72)
(46699, 7, 3.26)
(46649, 7, 3.26)
(45310, 7, 3.26)
(73300, 6, 2.79)
(70210, 6, 2.79)
(62100, 6, 2.79)
(97000, 5, 2.33)
(95000, 5, 2.33)
(71121, 5, 2.33)
(51100, 5, 2.33)
(50301, 5, 2.33)
(74203, 4, 1.86)
(73200, 4, 1.86)
(70220, 4, 1.86)
(70200, 4, 1.86)
(52100, 4, 1.86)
(51180, 4, 1.86)
(46820, 4, 1.86)
(46180, 4, 1.86)
(94999, 3, 1.4)
(94992, 3, 1.4)
(92400, 3, 1.4)
(91330, 3, 1.4)
(74142, 3, 1.4)
(74141, 3, 1.4)
(74131, 3, 1.4)
(72100, 3, 1.4)
(63910, 3, 1.4)
(63122, 3, 1.4)
(62200, 3, 1.4)
(62020, 3, 1.4)
(60310, 3, 1.4)
(52290, 3, 1.4)
(52250, 3, 1.4)
(51900, 3, 1.4)
(51810, 3, 1.4)
(51562, 3, 1.4)
(51541, 3, 1.4)
(51520, 3, 1.4)
(51130, 3, 1.4)
(47910, 3, 1.4)
(46900, 3, 1.4)
(46869, 3, 1.4)
(46841, 3, 1.4)
(46769, 3, 1.4)
(46741, 3, 1.4)
(46620, 3, 1.4)
(46130, 3, 1.4)
(45340, 3, 1.4)
(43299, 3, 1.4)
(43240

In [97]:
# Check if enterprises have multiple NACE codes
enterprises_multiple_nace = """
SELECT 
    activity.EntityNumber,
    COUNT(DISTINCT activity.NaceCode) as nace_count,
    GROUP_CONCAT(DISTINCT activity.NaceCode) as nace_codes
FROM activity
GROUP BY activity.EntityNumber
HAVING COUNT(DISTINCT activity.NaceCode) > 1
ORDER BY nace_count DESC
LIMIT 100
"""
execute_query(enterprises_multiple_nace)

('2.163.164.138', 748, '46699,4311001,96099,4753002,4291101,43390,27120,16220,4332004,4613001,4299001,25110,81220,2790011,2740203,2561005,4332031,7111102,2511001,2550102,382 ... (5108 characters truncated) ... 500,43990,43223,81230,4684102,4342901,4664301,3100104,4332006,4323001,38330,7020002,6812203,77224,4399003,4100101,6812101,46872,2551004,47640,4618035')
('2.173.586.292', 669, '2814001,63120,25290,161003,4614005,4322211,2893003,81290,7739901,4763001,4741001,74909,4730001,4322202,46620,43211,2512001,7733002,2561001,2825001,46 ... (4609 characters truncated) ... 001,46879,4323003,1611004,2599903,4755902,8123001,4684202,4324001,3100302,4342903,77221,31004,4683101,47552,4681002,4221903,2551002,46841,62100,70200')
('2.036.350.890', 605, '43350,43390,4614002,7120905,4391002,4399906,43320,6832101,43130,46150,68204,4673201,26300,82300,3109101,4941003,6820302,2223001,3101012,6810002,43120 ... (4167 characters truncated) ... 00901,68310,46832,25401,46831,22240,6812202,4683102,4399002,433500